<a href="https://colab.research.google.com/github/katrina906/CS6120-Summarization-Project/blob/main/abstractive_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install import-ipynb

In [2]:
from transformers import pipeline
import pandas as pd
import torch
import import_ipynb

In [3]:
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# load in functions from extract_summarization notebook
%cd "drive/MyDrive/Colab Notebooks"
from attention import *
%cd ..

/content/drive/MyDrive/Colab Notebooks
importing Jupyter notebook from attention.ipynb
/content/drive/MyDrive


In [ ]:
# load data
df = pd.read_pickle("/content/drive/MyDrive/data/cleaned_df.pkl")
df = df.sample(frac = 1, random_state = 123).head(10000) # so comparable to extractive 
#article = df.iloc[0].text

In [ ]:
# TODO split into train, validate, and test. supervised learning. validate for training, test for statistics

In [ ]:
df[['sentences', 'sentences_summary']] # TODO use cleaned text
df['text_seq'] = df.sentences.map(lambda row: ' '.join(row))
df['summary_seq'] = df.sentences_summary.map(lambda row: '_START_ ' + ' '.join(row) + ' _END_')

In [ ]:
from sklearn.model_selection import train_test_split
text_tr, text_test, summary_tr, summary_test = train_test_split(df['text_seq'].to_list(), df['summary_seq'].to_list(), test_size=0.1, random_state=0, shuffle=True) 
text_tr, text_val, summary_tr, summary_val = train_test_split(text_tr, summary_tr, test_size=0.1, random_state=0, shuffle=True) 

In [ ]:
max_len_text = 50 # TODO set max len to actual max len 

In [ ]:
def generate_token_sequences(text):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(list(text))
  seq = tokenizer.texts_to_sequences(list(text))
  seq = pad_sequences(seq,  maxlen=max_len_text, padding='post') 
  voc_size = len(tokenizer.word_index) + 1

  return seq, voc_size

In [ ]:
text_tr, text_tr_voc_size = generate_token_sequences(text_tr)
summary_tr, summary_tr_voc_size = generate_token_sequences(summary_tr)
text_val, text_val_voc_size = generate_token_sequences(text_val)
summary_val, summary_val_voc_size = generate_token_sequences(summary_val)

In [ ]:
from keras import backend as K 
K.clear_session() 
latent_dim = 500 # TODO tune?

In [ ]:
# Encoder 
encoder_inputs = Input(shape=(max_len_text,)) 
enc_emb = Embedding(text_tr_voc_size, latent_dim,trainable=True)(encoder_inputs) 

In [ ]:
#LSTM 1 
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True) 
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb) 

#LSTM 2 
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True) 
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1) 

#LSTM 3 
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True) 
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2) 

# TODO different numbers of layers

In [ ]:
# Set up the decoder. 
decoder_inputs = Input(shape=(None,)) 
dec_emb_layer = Embedding(summary_tr_voc_size, latent_dim,trainable=True) 
dec_emb = dec_emb_layer(decoder_inputs) 

In [ ]:
#LSTM using encoder_states as initial state
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True) 
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c]) 

#Attention Layer
attn_layer = AttentionLayer(name='attention_layer') 
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs]) 

# Concat attention output and decoder LSTM output 
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

#Dense layer
decoder_dense = TimeDistributed(Dense(summary_tr_voc_size, activation='softmax')) 
decoder_outputs = decoder_dense(decoder_concat_input) 

In [ ]:
# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs) 
model.summary()

In [ ]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy') # TODO try diff optimizers, loss

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1) # TODO look at documentation

In [ ]:
history=model.fit([text_tr,summary_tr[:,:-1]],
                  summary_tr.reshape(summary_tr.shape[0],summary_tr.shape[1], 1)[:,1:],
                  epochs=1, # 50
                  callbacks=[es],
                  batch_size=512, # 512
                  validation_data=([text_val,summary_val[:,:-1]], summary_val.reshape(summary_val.shape[0],summary_val.shape[1], 1)[:,1:]))
# https://keras.io/api/models/model_training_apis/#fit-method
# why is it using x and y as the x data?
# shifitng summary data in both cases?

In [ ]:
#summarizer = pipeline("summarization", framework = 'pt')
#summarized = summarizer(article, min_length=0, max_length=100) # max length is number of words/tokens
# max number of tokens can consider: 1024. Need to limit to first 1024
  # ok because best info tends to be at front of articles (and bcg documents bc pyramid principle)
  # potential extension is to fine tune with a higher max allowed